Importing libraries

In [389]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [390]:
SEED_VAL = 0
tf.__version__
tf.random.set_seed(SEED_VAL)

In [391]:
from ucimlrepo import fetch_ucirepo

# fetch dataset 
student_performance = fetch_ucirepo(id=320) 
  
# data (as pandas dataframes) 
X = student_performance.data.features.to_numpy()
y_raw = student_performance.data.targets.to_numpy()
#y_raw = y_raw[:, 2]  # I'm using only the 3rd target column (final grade)

### Converting y to a 1D array and Encode it.

In [392]:
y = y_raw[:, 2].ravel()
#print(sorted(set(y)))
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y = le.fit_transform(y)

In [393]:
## NEW! OneHotEncode y column

In [394]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(sparse_output=False)
onehotencoder.fit(np.array([i for i in range(21)]).reshape(-1,1))
# y_encoded = onehotencoder.fit_transform(y.reshape(-1, 1))
# y_encoded.shape
# y = y_encoded
y_encoded = onehotencoder.transform(y.reshape(-1,1))
y = y_encoded
#print(yd[0])

## Encoding feature columns

In [395]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le = LabelEncoder()

# encode the first column (school)
X[:, 0] = le.fit_transform(X[:, 0])
X[:, 1] = le.fit_transform(X[:, 1])   # gender

# encode the 4th column. Rural or Urban
X[:, 3] = le.fit_transform(X[:, 3])    # address type 
X[:, 4] = le.fit_transform(X[:, 4])    # family size
X[:, 5] = le.fit_transform(X[:, 5])    # family cohabitation status

print("X shape before oneHot ", X.shape)  # Todo: remove this

# 9th column (mother's job) is nominal
onehotencoder = OneHotEncoder(categories='auto', sparse_output=False)    # set to false to return ndarry instead of scipy.sparse._csr.csr_matrix
col_9_encoded = onehotencoder.fit_transform(X[:, 8].reshape(-1, 1))
print("new dim added: ", col_9_encoded.shape)
X = np.concatenate((X[:,:8], col_9_encoded, X[:, 9:]), axis=1)  # add/concat the RHS array as a new column(s). Now we have 34cols
# at this point, col9 at idx8 has extended to indexes 8,9,10,11,12 due to the new encoded indexes
print(f"X's shape after mjob5: {X.shape}")

# encoding father's job column. Originally col idx9, now idx13
col_fjob_encoded = onehotencoder.fit_transform(X[:, 13].reshape(-1, 1))
print("new dim added: ", col_fjob_encoded.shape)
X = np.concatenate((X[:,:13], col_fjob_encoded, X[:, 14:]), axis=1)  # add/concat the RHS array as 5 new column(s)
print(f"X's shape after fjob5: {X.shape}")

# encoding the reason column
col_reason_encoded = onehotencoder.fit_transform(X[:, 18].reshape(-1, 1))
print("new dim added: ", col_reason_encoded.shape)
X = np.concatenate((X[:,:18], col_reason_encoded, X[:, 19:]), axis=1)  # add/concat the RHS array as 4 new column(s)
print(f"X's shape after reason4: {X.shape}")

# encoding the guardian column
col_guardian_encoded = onehotencoder.fit_transform(X[:, 22].reshape(-1, 1))
print("new guard cols added: ", col_guardian_encoded.shape)
X = np.concatenate((X[:,:22], col_guardian_encoded, X[:, 23:]), axis=1)  # add/concat the RHS array as 3 new column(s)
print(f"X's shape after guardian3: {X.shape}")

# encoding the remaining binary columns
for col in range(28, 36):
    X[:, col] = le.fit_transform(X[:, col]) 

print(f"X's new shape: {X.shape}")
print(X[0])

X shape before oneHot  (649, 30)
new dim added:  (649, 5)
X's shape after mjob5: (649, 34)
new dim added:  (649, 5)
X's shape after fjob5: (649, 38)
new dim added:  (649, 4)
X's shape after reason4: (649, 41)
new guard cols added:  (649, 3)
X's shape after guardian3: (649, 43)
X's new shape: (649, 43)
[0 0 18 1 0 0 4 4 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0
 0.0 1.0 0.0 2 2 0 1 0 0 0 1 1 0 0 4 3 4 1 1 3 4]


In [396]:
# adding extra output columns to X
G1,G2 = y_raw[:,0].reshape(-1,1), y_raw[:,1].reshape(-1,1)
print(X.shape)
X = np.concatenate((X, G1, G2), axis=1)
print(X.shape)

(649, 43)
(649, 45)


## Splitting the dataset into the Training and Test sets

In [397]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling
we scale the features so they're in the same range

In [398]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

### Initializing the ANN

In [399]:
#ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [400]:
#ann.add(tf.keras.layers.Dense(units=16, activation='relu'))

### Adding the second hidden layer

In [401]:
#ann.add(tf.keras.layers.Dense(units=8, activation='relu'))

### Adding the output layer

In [402]:
#ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training The ANN

### Compiling the ANN

In [403]:
#ann.fit(X_train, y_train, batch_size = 32, epochs = 100, shuffle=False)

In [404]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Running GridSearch

In [405]:
# print("Running gridsearch...")

# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# # Function to create model, required for KerasClassifier
# def create_modelOld(optimizer='adam', dropout_rate=0.0):
#     model = Sequential()
#     model.add(Dense(64, input_dim=45, activation='relu'))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(17, activation='softmax'))
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
#     return model

# def create_model(optimizer='adam', init='uniform'):
#     model = Sequential()
#     # Input layer with 45 features
#     model.add(Dense(64, input_dim=45, kernel_initializer=init, activation='relu'))
#     model.add(Dense(64, kernel_initializer=init, activation='relu'))
#     # Output layer with 21 classes and softmax activation
#     model.add(Dense(21, kernel_initializer=init, activation='softmax'))
    
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#     return model
    
# #model = KerasClassifier(build_fn=create_model, verbose=0)

# # Define the grid of hyperparameters to search
# param_grid = {
#     'batch_size': [10, 20, 40],
#     'epochs': [50, 100],
#     'optimizer': ['adam', 'rmsprop'],
#     'dropout_rate': [0.0, 0.2, 0.4]
# }

# print("Gridsearch done!")

## Building a custom ANN for multiclass problem

In [413]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Build the ANN model
input_dim = 45
output_dim = 21
# model = Sequential([
#     Dense(64, activation='relu', input_shape=(input_dim,)),
#     Dense(64, activation='relu'),
#     Dense(output_dim, activation='softmax')  # 21 neurons for 21 classes
# ])

from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

# model = Sequential([
#     Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
#     Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
#     Dense(output_dim, activation='softmax')
# ])



# model = Sequential([
#     Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
#     Dropout(0.2),  # Add dropout after first layer (50% neurons dropped)
#     Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
#     Dropout(0.2),  # Add dropout after second layer
#     Dense(output_dim, activation='softmax')
# ])

# more hidden layers
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),  # Added hidden layer
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),  # Added another hidden layer
    Dense(output_dim, activation='softmax')
])


# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])



Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.0071 - loss: 5.0166 - val_accuracy: 0.0673 - val_loss: 4.7732
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1181 - loss: 4.7108 - val_accuracy: 0.1058 - val_loss: 4.5461
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1690 - loss: 4.4909 - val_accuracy: 0.1154 - val_loss: 4.3268
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1746 - loss: 4.2598 - val_accuracy: 0.0962 - val_loss: 4.0835
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1845 - loss: 4.0057 - val_accuracy: 0.1538 - val_loss: 3.8554
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1972 - loss: 3.7793 - val_accuracy: 0.1346 - val_loss: 3.6654
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2035 - loss: 3.5823 - val_accuracy: 0.1442 - val_loss: 3.5179
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2456 - loss: 3.4136 - val_accuracy: 0.1538 - 

In [414]:
# Evaluate the model
loss_train, accuracy_train = model.evaluate(X_train, y_train)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy_train:.2f}")
print(f"Test Accuracy: {accuracy:.2f}")

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6137 - loss: 1.9415 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2718 - loss: 2.5804 
Test Accuracy: 0.58
Test Accuracy: 0.32


### Training the ANN on the Training set

## Model Predictions and Evaluations

### Predicting Insample test results

In [408]:
# y_pred_ins = ann.predict(X_train)
# y_pred_ins = (y_pred_ins > 0.5).astype("int")

In [409]:
# # Getting the accuracy score
# from sklearn.metrics import confusion_matrix, accuracy_score
# cm = confusion_matrix(y_train, y_pred_ins)
# #print(cm)
# accuracy_score(y_train, y_pred_ins)

### Out-Sample Prediction

In [410]:
# y_pred = ann.predict(X_test)
# y_pred = (y_pred > 0.5).astype("int")
# #print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [411]:
# cm = confusion_matrix(y_test, y_pred)
# #print(cm)
# accuracy_score(y_test, y_pred)